In [28]:
import cv2
import numpy
import typing
import logging
import gevent.monkey
import gevent
import sys
args = sys.argv

from utils import create_folder, get_logger
logger: logging.Logger = get_logger(__name__)
gevent.monkey.patch_all()

def gaussian_2d_convolution(size:int)-> numpy.ndarray:
    gussian_1d = cv2.getGaussianKernel(ksize=size, sigma=1.0)
    # logger.info(f"size:{size}, gussian_1d:{gussian_1d}")
    return numpy.outer(a=gussian_1d, b=gussian_1d)

def haar_masks(scale: int):
    mask_arr = []
    for i in range(scale):
        size = 2 ** i  # 每个 mask 的尺寸是 2^i x 2^i
        mask = numpy.array([[1 / (size ** 2)] * size] * size, dtype=float)
        mask_arr.append(mask)
    return mask_arr


convolut_kernel_dict = {
    "sobel_kernel_x_3":numpy.array(object=[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=float),
    "sobel_kernel_y_3":numpy.array(object=[[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=float),
    "sobel_kernel_x_5": numpy.array(object=[[-1,-2, 0, 2, 1], [-2, -3, 0 ,3 ,2], [0, 0, 0 , 0, 0], [2,3,0,-3,-2], [1,2,0,-2,-1]], dtype=float),
    "sobel_kernel_y_5": numpy.array(object=[[-1,-4, -6, -4, -1], [-2, -8, -12 ,-8 , -2], [0, 0, 0 , 0, 0], [2,8,12,8,2], [1,4,6,4,1]], dtype=float),
    "gaussian": gaussian_2d_convolution
}
image_path = "/Users/admin/Desktop/EE5371/CA1/einstein.png"
SOBEL_FOLDER_NAME = "sobel"
SOBEL_FOLDER_NAME2 = "sobel2"
GUSSIAN_FOLDER_NAME = "gaussian"
# cv.IMREAD_COLOR cv.IMREAD_GRAYSCALE
origin_image: cv2.typing.MatLike = cv2.imread(filename=image_path, flags=cv2.IMREAD_GRAYSCALE)
logger.info(f"origin_image:{origin_image}")

# sobel kernel
# use  cv2.filter2D api
sobel_size_arr: typing.List[int] = [3, 5]
create_folder(folder_name=SOBEL_FOLDER_NAME)
create_folder(folder_name=SOBEL_FOLDER_NAME2)
for sobel_size in sobel_size_arr:
    sobel_x_result:cv2.typing.MatLike = cv2.filter2D(src=origin_image, 
                                                     ddepth=-1, 
                                                     kernel=convolut_kernel_dict.get(f"sobel_kernel_x_{sobel_size}"))
    sobel_y_result:cv2.typing.MatLike = cv2.filter2D(src=origin_image, 
                                                     ddepth=-1, 
                                                     kernel=convolut_kernel_dict.get(f"sobel_kernel_y_{sobel_size}"))
    sobel_combo_result:cv2.typing.MatLike = cv2.addWeighted(src1=sobel_x_result, alpha=0.5, src2=sobel_y_result, beta=1-0.5, gamma=1.0)
    create_folder(folder_name=f"{SOBEL_FOLDER_NAME}/size{sobel_size}")
    cv2.imwrite(f"{SOBEL_FOLDER_NAME}/size{sobel_size}/sobel_x_result.png", sobel_x_result)
    cv2.imwrite(f"{SOBEL_FOLDER_NAME}/size{sobel_size}/sobel_y_result.png", sobel_y_result)
    cv2.imwrite(f"{SOBEL_FOLDER_NAME}/size{sobel_size}/sobel_combo_result.png", sobel_combo_result)

    # use cv2.Sobel api
    sobel_x_result:cv2.typing.MatLike = cv2.Sobel(src=origin_image, ddepth=-1, dx=1, dy=0, ksize=sobel_size) 
    sobel_y_result:cv2.typing.MatLike = cv2.Sobel(src=origin_image, ddepth=-1, dx=0, dy=1, ksize=sobel_size) 
    sobel_combo_result:cv2.typing.MatLike = cv2.Sobel(src=origin_image, ddepth=-1, dx=1, dy=1, ksize=sobel_size)
    # the brightness is too low
    create_folder(folder_name=f"{SOBEL_FOLDER_NAME2}/size{sobel_size}")
    cv2.imwrite(f"{SOBEL_FOLDER_NAME2}/size{sobel_size}/sobel_x_result.png", sobel_x_result)
    cv2.imwrite(f"{SOBEL_FOLDER_NAME2}/size{sobel_size}/sobel_y_result.png", sobel_y_result)
    cv2.imwrite(f"{SOBEL_FOLDER_NAME2}/size{sobel_size}/sobel_combo_result.png", sobel_combo_result)

# Gaussian kernel

image_height, image_width = origin_image.shape
logger.info(f"shape:{origin_image.shape}")
max_gaussian_size:int = image_width // 16
create_folder(folder_name=GUSSIAN_FOLDER_NAME)

def guassian_convolution(image: cv2.typing.MatLike, guassian_size: int, group:int = 5 ):
    create_folder(folder_name=f"{GUSSIAN_FOLDER_NAME}/size{guassian_size}")
    scale_factor = 1
    # it's better to use √(1.6^2 - 0.5^2)
    sigma = 1.52
    resized_image = image
    for it in range(1,group):
        new_width = int(image.shape[1] * scale_factor)
        new_height = int(image.shape[0] * scale_factor)
        scale_factor /= 2
        resized_image = cv2.resize(resized_image, (new_width, new_height))
        guassian_result:cv2.typing.MatLike = cv2.GaussianBlur(src=resized_image, 
                                                              ksize=(guassian_size, guassian_size), 
                                                              sigmaX=sigma, sigmaY=sigma, 
                                                              borderType=cv2.BORDER_ISOLATED)
        sigma *=2 
        cv2.imwrite(f"{GUSSIAN_FOLDER_NAME}/size{guassian_size}/guassian_result_it{it}.png", guassian_result)
# log2(min(M,N)) -3
o = int(numpy.log2(min(image_height, image_width))) - 3
for guassian_size in range(3, max_gaussian_size, 2):
    coroutine = gevent.spawn(guassian_convolution, **{
        "image": origin_image,
        "guassian_size": guassian_size,
        "group": o
    })





2023-09-25 02:24:42,022 - __main__ - INFO - origin_image:[[ 24  23  23 ...  28  29  30]
 [ 23  23  23 ...  29  30  32]
 [ 23  23  22 ...  30  30  32]
 ...
 [  6   6   6 ... 115 114 112]
 [ 13  13  12 ... 114 114 112]
 [  7   9   9 ... 112 112 110]]
2023-09-25 02:24:42,022 - __main__ - INFO - origin_image:[[ 24  23  23 ...  28  29  30]
 [ 23  23  23 ...  29  30  32]
 [ 23  23  22 ...  30  30  32]
 ...
 [  6   6   6 ... 115 114 112]
 [ 13  13  12 ... 114 114 112]
 [  7   9   9 ... 112 112 110]]


2023-09-25 02:24:42,022 - __main__ - INFO - origin_image:[[ 24  23  23 ...  28  29  30]
 [ 23  23  23 ...  29  30  32]
 [ 23  23  22 ...  30  30  32]
 ...
 [  6   6   6 ... 115 114 112]
 [ 13  13  12 ... 114 114 112]
 [  7   9   9 ... 112 112 110]]
2023-09-25 02:24:42,022 - __main__ - INFO - origin_image:[[ 24  23  23 ...  28  29  30]
 [ 23  23  23 ...  29  30  32]
 [ 23  23  22 ...  30  30  32]
 ...
 [  6   6   6 ... 115 114 112]
 [ 13  13  12 ... 114 114 112]
 [  7   9   9 ... 112 112 110]]
2023-09-25 02:24:42,022 - __main__ - INFO - origin_image:[[ 24  23  23 ...  28  29  30]
 [ 23  23  23 ...  29  30  32]
 [ 23  23  22 ...  30  30  32]
 ...
 [  6   6   6 ... 115 114 112]
 [ 13  13  12 ... 114 114 112]
 [  7   9   9 ... 112 112 110]]
2023-09-25 02:24:42,022 - __main__ - INFO - origin_image:[[ 24  23  23 ...  28  29  30]
 [ 23  23  23 ...  29  30  32]
 [ 23  23  22 ...  30  30  32]
 ...
 [  6   6   6 ... 115 114 112]
 [ 13  13  12 ... 114 114 112]
 [  7   9   9 ... 112 112 110]]
2023